In [71]:
!pip install -q geopy

In [1]:
import pandas as pd 
import time 
#geocode cities 
from geopy.geocoders import Nominatim

#to get combination
from itertools import product

#straight line distance conversion 
import math
from math import sin, cos, sqrt, atan2, radians
import numpy as np


In [125]:
capital_dic={
    'Alabama': 'Montgomery',
    'Alaska': 'Juneau',
    'Arizona':'Phoenix',
    'Arkansas':'Little Rock',
    'California': 'Sacramento',
    'Colorado':'Denver',
    'Connecticut':'Hartford',
    'Delaware':'Dover',
    'Florida': 'Tallahassee',
    'Georgia': 'Atlanta',
    'Hawaii': 'Honolulu',
    'Idaho': 'Boise',
    'Illinios': 'Springfield',
    'Indiana': 'Indianapolis',
    'Iowa': 'Des Monies',
    'Kansas': 'Topeka',
    'Kentucky': 'Frankfort',
    'Louisiana': 'Baton Rouge',
    'Maine': 'Augusta',
    'Maryland': 'Annapolis',
    'Massachusetts': 'Boston',
    'Michigan': 'Lansing',
    'Minnesota': 'St. Paul',
    'Mississippi': 'Jackson',
    'Missouri': 'Jefferson City',
    'Montana': 'Helena',
    'Nebraska': 'Lincoln',
    'Neveda': 'Carson City',
    'New Hampshire': 'Concord',
    'New Jersey': 'Trenton',
    'New Mexico': 'Santa Fe',
    'New York': 'Albany',
    'North Carolina': 'Raleigh',
    'North Dakota': 'Bismarck',
    'Ohio': 'Columbus',
    'Oklahoma': 'Oklahoma City',
    'Oregon': 'Salem',
    'Pennsylvania': 'Harrisburg',
    'Rhoda Island': 'Providence',
    'South Carolina': 'Columbia',
    'South Dakoda': 'Pierre',
    'Tennessee': 'Nashville',
    'Texas': 'Austin',
    'Utah': 'Salt Lake City',
    'Vermont': 'Montpelier',
    'Virginia': 'Richmond',
    'Washington': 'Olympia',
    'West Virginia': 'Charleston',
    'Wisconsin': 'Madison',
    'Wyoming': 'Cheyenne'  
} # create a dictionary, key is the state and value is the capital

capitals = pd.DataFrame.from_dict(capital_dic, orient='index', columns=["capital"])


In [126]:
capitals["state"] = capitals.index
capitals["forsearch"] = capitals[['capital', 'state']].apply(lambda x: ' '.join(x), axis=1)

In [127]:
capitals.head()

,capital,state,forsearch
Alabama,Montgomery,Alabama,Montgomery Alabama
Alaska,Juneau,Alaska,Juneau Alaska
Arizona,Phoenix,Arizona,Phoenix Arizona
Arkansas,Little Rock,Arkansas,Little Rock Arkansas
California,Sacramento,California,Sacramento California


In [123]:
def get_geocode(city):
    geolocator = Nominatim(user_agent='myapplication')
    location = geolocator.geocode(city)
    lat = location.raw["lat"]
    lon = location.raw["lon"]
    return lat+" "+lon

In [124]:
data1 = capitals[0:10]
data2 = capitals[10:20]
data3 = capitals[20:30]
data4 = capitals[30:40]
data5 = capitals[40:50]
data_arr = [data1["forsearch"], data2["forsearch"], data3["forsearch"], data4["forsearch"], data5["forsearch"]]


In [125]:
geo_arr = []

for sub in data_arr:   
    for x in sub:
        try:  
            geo_arr.append(get_geocode(x))
            time.sleep(1)
        except:
            geo_arr.append("na")
            time.sleep(1)

In [126]:
len(geo_arr)

50

In [128]:
capitals["geocode"] = geo_arr
capitals.to_csv("geocoded_capital.csv", index = False)


### In Excel, we process the missing geocodes and standardize geocodes as well 

In [130]:
capitals_geo = pd.read_csv("geocoded_capital.csv"); capitals_geo.head(2)

,capital,state,lat,lon
0,Montgomery,Alabama,32.366966,-86.300648
1,Juneau,Alaska,58.301950,-134.419734


### get distance between any two capitals

In [147]:
#to get combination of cities, join all text and ...
capitals_geo["lat"] = capitals_geo["lat"].astype(str)
capitals_geo["lon"] = capitals_geo["lon"].astype(str)

combination = list(capitals_geo[['capital', 'state', "lat", "lon"]].apply(lambda x: ','.join(x), axis=1))
# combination_df = pd.DataFrame({"text": combination})
combination[0:2]

['Montgomery,Alabama,32.3669656,-86.3006485',
 'Juneau,Alaska,58.30194960000001,-134.419734']

In [149]:
df = pd.DataFrame(data=list(product(combination, combination)), columns=['origin','destination'])
df.head(5)

,origin,destination
0,"Montgomery,Alabama,32.3669656,-86.3006485","Montgomery,Alabama,32.3669656,-86.3006485"
1,"Montgomery,Alabama,32.3669656,-86.3006485","Juneau,Alaska,58.30194960000001,-134.419734"
2,"Montgomery,Alabama,32.3669656,-86.3006485","Phoenix,Arizona,33.448586600000006,-112.0773456"
3,"Montgomery,Alabama,32.3669656,-86.3006485","Little Rock,Arkansas,34.7464809,-92.2895948"
4,"Montgomery,Alabama,32.3669656,-86.3006485","Sacramento,California,38.5815719,-121.49439960..."


In [150]:
df.to_csv("combination.csv", index=False)

### calculate straght Distance

In [110]:
distances = pd.read_csv("combination.csv") 
print(distances.shape)
distances.head(2)

(2450, 8)


,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon
0,Montgomery,Alabama,32.366966,-86.300648,Juneau,Alaska,58.301950,-134.419734
1,Montgomery,Alabama,32.366966,-86.300648,Phoenix,Arizona,33.448587,-112.077346


In [111]:
distances["lat1"]=np.radians(distances["origin_lat"])
# approximate radius of earth in km
R = 6373.0*0.621371

distances["lat1"] = np.radians(distances["origin_lat"])
distances["lon1"] = np.radians(distances["origin_lon"])
distances["lat2"] = np.radians(distances["destination_lat"])
distances["lon2"] = np.radians(distances["destination_lon"])

distances["dlon"] = distances["lon2"] - distances["lon1"]
distances["dlat"] = distances["lat2"] - distances["lat1"]

distances["a"] = np.sin(distances["dlat"] / 2)**2 + np.cos(distances["lat1"]) * np.cos(distances["lat2"]) * np.sin(distances["dlon"] / 2)**2
distances["c"] = 2 * np.arctan2(np.sqrt(distances["a"]), np.sqrt(1 - distances["a"]))
distances["distance_mile"] = R * distances["c"]


In [112]:
distances.drop(["lat1", "lon1", "lat2", "lon2", "dlon", "dlat", "a", "c"], axis=1, inplace=True)
distances.head()

,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile
0,Montgomery,Alabama,32.366966,-86.300648,Juneau,Alaska,58.301950,-134.419734,2851.459408
1,Montgomery,Alabama,32.366966,-86.300648,Phoenix,Arizona,33.448587,-112.077346,1493.746102
2,Montgomery,Alabama,32.366966,-86.300648,Little Rock,Arkansas,34.746481,-92.289595,382.045028
3,Montgomery,Alabama,32.366966,-86.300648,Sacramento,California,38.581572,-121.494400,2014.022091
4,Montgomery,Alabama,32.366966,-86.300648,Denver,Colorado,39.739236,-104.984862,1158.599319


In [113]:
print(distances.shape)
distances_sorted = distances.sort_values(by="distance_mile");distances_sorted.head()

(2450, 9)


,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile
244,Sacramento,California,38.581572,-121.494400,Cheyenne,Wyoming,38.040803,-121.336657,38.341730
2405,Cheyenne,Wyoming,38.040803,-121.336657,Sacramento,California,38.581572,-121.494400,38.341730
1017,Boston,Massachusetts,42.360253,-71.058291,Providence,Rhoda Island,41.824877,-71.413618,41.246682
1882,Providence,Rhoda Island,41.824877,-71.413618,Boston,Massachusetts,42.360253,-71.058291,41.246682
938,Annapolis,Maryland,38.978640,-76.492786,Dover,Delaware,39.158168,-75.524368,53.426284


In [114]:
end = 2450
values = []
for i in range(0,end):
    if i%2 == 0:
        values.append(0)
    else: 
        values.append(1)


In [115]:
distances_sorted["filter"] = values;distances_sorted.tail(2)

,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile,filter
507,Honolulu,Hawaii,21.325079,-157.846644,Augusta,Maine,44.310583,-69.779663,5110.88414,0
892,Augusta,Maine,44.310583,-69.779663,Honolulu,Hawaii,21.325079,-157.846644,5110.88414,1


In [116]:
distance_unique = distances_sorted[distances_sorted["filter"] == 1]
distance_unique.drop(["filter"], axis=1, inplace = True)
print(len(distance_unique))

1225


In [117]:
distance_unique.to_csv("distances.csv", index=False)
#removed duplicate distance in excel, A-B and B-A reduced to 1 distance

## Trips!!


In [2]:
distance_unique = pd.read_csv("distances.csv")

In [3]:
distance_unique.head()

,origin_city,origin_state,origin_lat,origin_lon,destination_city,destination_state,destination_lat,destination_lon,distance_mile
0,Cheyenne,Wyoming,38.040803,-121.336657,Sacramento,California,38.581572,-121.494400,38.341730
1,Providence,Rhoda Island,41.824877,-71.413618,Boston,Massachusetts,42.360253,-71.058291,41.246682
2,Dover,Delaware,39.158168,-75.524368,Annapolis,Maryland,38.978640,-76.492786,53.426284
3,Concord,New Hampshire,43.207178,-71.537476,Boston,Massachusetts,42.360253,-71.058291,63.380937
4,Providence,Rhoda Island,41.824877,-71.413618,Hartford,Connecticut,41.764582,-72.690855,65.944394


In [189]:
start_city = "Sacramento"
route = [start_city]
total_distance = 0
# route[-1]

def optimize_distance(city):
    nth = 1
    global total_distance
    min_distance = distance_unique[distance_unique["origin_city"] == city]["distance_mile"].min()
    min_distance_city = distance_unique[distance_unique["distance_mile"] == min_distance]["destination_city"].values[0]
    while min_distance_city in route: 
        #take nth smallest distance to avoid duplicate route
        nth += 1 
        min_distance_row = distance_unique[distance_unique["origin_city"] == city].nsmallest(nth, 'distance_mile', keep='last').iloc[nth-1,]
        min_distance = min_distance_row["distance_mile"]
        min_distance_city = min_distance_row["destination_city"]
        
    route.append(min_distance_city)
    total_distance += min_distance    
        

In [190]:
# we will plot AK and HI mannually 
for i in range(0,47): 
    optimize_distance(route[-1])

print(route)
print(total_distance)


['Sacramento', 'Phoenix', 'Santa Fe', 'Salt Lake City', 'Helena', 'Bismarck', 'Des Monies', 'Topeka', 'Oklahoma City', 'Little Rock', 'Jefferson City', 'Springfield', 'Madison', 'Columbus', 'Frankfort', 'Charleston', 'Richmond', 'Annapolis', 'Trenton', 'Providence', 'Boston', 'Hartford', 'Albany', 'Montpelier', 'Harrisburg', 'Dover', 'Concord', 'Lansing', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Jackson', 'Austin', 'Indianapolis', 'St. Paul', 'Lincoln', 'Pierre', 'Boise', 'Salem', 'Olympia', 'Carson City', 'Denver', 'Baton Rouge', 'Cheyenne', 'Augusta']
20439.376777827387


In [191]:
len(route)

48

In [192]:
# Alaska: Juneau and Hawaii: Holonunu
#insert Holonunu Hawaii between 'Sacramento' and 'Phoenix'
#insert Juneau Alaska 'Salem' and 'Olympia'
place_Sacramento = route.index("Sacramento")+1
route.insert(place_Sacramento,'Holonunu')
place_Salem = route.index("Salem")+1
route.insert(place_Salem,'Juneau')

#adding distance between Sacramento to Holonunu to Phoenix THEN Salem to Juneau to Olympia
total_distance = total_distance + 2,460.70 + 2,907.38 + 1041 + 913

In [193]:
print(total_distance)
print(route)

(20441.376777827387, 462.7, 2861.38)
['Sacramento', 'Holonunu', 'Phoenix', 'Santa Fe', 'Salt Lake City', 'Helena', 'Bismarck', 'Des Monies', 'Topeka', 'Oklahoma City', 'Little Rock', 'Jefferson City', 'Springfield', 'Madison', 'Columbus', 'Frankfort', 'Charleston', 'Richmond', 'Annapolis', 'Trenton', 'Providence', 'Boston', 'Hartford', 'Albany', 'Montpelier', 'Harrisburg', 'Dover', 'Concord', 'Lansing', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Columbia', 'Raleigh', 'Jackson', 'Austin', 'Indianapolis', 'St. Paul', 'Lincoln', 'Pierre', 'Boise', 'Salem', 'Juneau', 'Olympia', 'Carson City', 'Denver', 'Baton Rouge', 'Cheyenne', 'Augusta']
